### Inicialização da API do Cartola FC e Configuração do Pandas

In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-05-07 11:58:46,931 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


### Organização dos Grupos da Fase 1 da Libertadores e Exportação para Excel

In [2]:
# Dicionário com ID -> Nome do time (gerado anteriormente)
nomes_por_id = {
    18661583: 'pura bucha /botafogo',
    3914981: 'BORGES ITAQUI F.C.',
    20696550: 'Dom Camillo68',    
    117598: 'A Lenda Super Vasco F.c',    
    17887202: 'HS SPORTS F.C',
    47543456: 'Tabajara de Inhaua FC2', 
    18223508: 'Rolo Compressor ZN',
    44810918: 'lsauer fc',
    479510: 'TEAM LOPES 99',
    3851966: 'cartola scheuer',
    24468241: 'Grêmio imortal 37',
    44574236: 'Analove10 ITAQUI GRANDE!!',
    1273719: 'Texas Club 2025',
    13707047: 'Super Vasco f.c',
    212042: 'Tatols Beants F.C',
    18642587: 'Fedato Futebol Clube',
    3246608: 'Real SCI',
    14369: 'ITAQUI F. C.',
    6148913: 'E.C. Bororé',
    528730: 'Gremiomaniasm',
    29228373: 'seralex',
    25311459: 'FC Los Castilho',
    49128587: 'TORRESMO COM PINGA',
    50158297: 'Lá do Itaqui',
    13951133: 'KING LEONN',
    49180400: 'TATITTA FC',
    36359: 'KillerColorado',
    159074: 'F.C. Rei Das Copas',
    18344271: 'Laranjja Mecannica',
    25751748: 'MauHumor F.C.',
    186283: 'FBC Colorado',
    18421230: 'Gig@ntte'
}

# Lista com todos os dados que você passou
dados_torneio = [
    # Grupo A
    ("Grupo A", 18661583, "Botafogo", "Gabriel"),
    ("Grupo A", 3914981, "Estudiantes", "Elvis"),
    ("Grupo A", 20696550, "Universidade Católica do Chile", "Camilo"),    
    ("Grupo A", 117598, "Carabobo", "Júlio Cesar"),    
    # Grupo B
    ("Grupo B", 17887202, "River Plate", "Henrique"),
    ("Grupo B", 47543456, "Independiente del Valle", "Ivanildo"),
    ("Grupo B", 18223508, "Universitário", "Lucas"),  # mesmo ID, último prevalece
    ("Grupo B", 44810918, "Barcelona", "Sauer"),
    # Grupo C
    ("Grupo C", 479510, "Flamengo", "Marllon"),
    ("Grupo C", 3851966, "LDU", "Paulo Augusto"),
    ("Grupo C", 24468241, "Deportivo Táchira", "Rodrigo"),
    ("Grupo C", 44574236, "Central Córdoba", "Ananias"),
    # Grupo D
    ("Grupo D", 1273719, "São Paulo", "Fumaco"),
    ("Grupo D", 13707047, "Libertad", "Boka"),
    ("Grupo D", 212042, "Talleres", "Elemar"),
    ("Grupo D", 18642587, "Alianza Lima", "Fedato"),
    # Grupo E
    ("Grupo E", 3246608, "Racing", "Diego"),
    ("Grupo E", 14369, "Colo-Colo", "Cassiano"),
    ("Grupo E", 6148913, "Fortaleza", "Moodi"),
    ("Grupo E", 528730, "Bucaramanga", "Márcio"),
    # Grupo F
    ("Grupo F", 29228373, "Atlético Nacional", "Alex"),
    ("Grupo F", 25311459, "Nacional", "Castilho"),
    ("Grupo F", 49128587, "Bahia", "Marquinho"),
    ("Grupo F", 50158297, "Internacional", "Gerônimo"),
    # Grupo G
    ("Grupo G", 13951133, "Palmeiras", "Leon"),
    ("Grupo G", 49180400, "Bolívar", "Tatita"),
    ("Grupo G", 36359, "Sporting Cristal", "Ronaldo"),
    ("Grupo G", 159074, "Cerro Porteño", "Ricardo"),
    # Grupo H
    ("Grupo H", 18344271, "Peñarol", "Roberto"),
    ("Grupo H", 25751748, "Olimpia", "André"),
     ("Grupo H", 186283, "Velez Sarsfield", "Fernando"),
    ("Grupo H", 18421230, "San Antônio Bulo Bulo", "Adriano"),   
]

# Criar DataFrame base
df_torneio = pd.DataFrame(dados_torneio, columns=["Grupo", "ID do Time", "Clube Representado", "Participante"])

# Adicionar Nome do Time usando o dicionário
df_torneio["Nome do Time"] = df_torneio["ID do Time"].map(nomes_por_id)

# Adicionar ID no Grupo
df_torneio["ID no Grupo"] = df_torneio.groupby("Grupo").cumcount() + 1
df_torneio["ID no Grupo"] = df_torneio["ID no Grupo"].astype(str) + "_" + df_torneio["Grupo"].str[-1]

# Reorganizar colunas
df_torneio = df_torneio[["Grupo", "ID do Time", "Nome do Time", "Clube Representado", "Participante", "ID no Grupo"]]

df_liberta_grupo_A = df_torneio[df_torneio["Grupo"] == "Grupo A"]
df_liberta_grupo_B = df_torneio[df_torneio["Grupo"] == "Grupo B"]
df_liberta_grupo_C = df_torneio[df_torneio["Grupo"] == "Grupo C"]
df_liberta_grupo_D = df_torneio[df_torneio["Grupo"] == "Grupo D"]
df_liberta_grupo_E = df_torneio[df_torneio["Grupo"] == "Grupo E"]
df_liberta_grupo_F = df_torneio[df_torneio["Grupo"] == "Grupo F"]
df_liberta_grupo_G = df_torneio[df_torneio["Grupo"] == "Grupo G"]
df_liberta_grupo_H = df_torneio[df_torneio["Grupo"] == "Grupo H"]


# Lista de grupos
grupos = {
    "Grupo A": df_liberta_grupo_A,
    "Grupo B": df_liberta_grupo_B,
    "Grupo C": df_liberta_grupo_C,
    "Grupo D": df_liberta_grupo_D,
    "Grupo E": df_liberta_grupo_E,
    "Grupo F": df_liberta_grupo_F,
    "Grupo G": df_liberta_grupo_G,
    "Grupo H": df_liberta_grupo_H,
}

# # Caminho do arquivo
# caminho_excel = "grupos_fase1_libertadores.xlsx"

# # Salvar em Excel com abas separadas
# with pd.ExcelWriter(caminho_excel) as writer:
#     for nome_grupo, df_grupo in grupos.items():
#         df_grupo.to_excel(writer, sheet_name=nome_grupo, index=False)


# Exibir resultado
# display(df_liberta_grupo_A)
# display(df_liberta_grupo_B)
# display(df_liberta_grupo_C)
# display(df_liberta_grupo_D)
# display(df_liberta_grupo_E)
display(df_liberta_grupo_F)
# display(df_liberta_grupo_G)
# display(df_liberta_grupo_H)

,Grupo,ID do Time,Nome do Time,Clube Representado,Participante,ID no Grupo
20,Grupo F,29228373,seralex,Atlético Nacional,Alex,1_F
21,Grupo F,25311459,FC Los Castilho,Nacional,Castilho,2_F
22,Grupo F,49128587,TORRESMO COM PINGA,Bahia,Marquinho,3_F
23,Grupo F,50158297,Lá do Itaqui,Internacional,Gerônimo,4_F


### Definição dos Confrontos das 6 Rodadas da Fase de Grupos da Libertadores



In [3]:
# Lista dos confrontos da 1ª rodada
confrontos_1a_rodada = [
    # Grupo A
    ("Grupo A", "4_A", "2_A"),
    ("Grupo A", "3_A", "1_A"),

    # Grupo B
    ("Grupo B", "4_B", "2_B"),
    ("Grupo B", "3_B", "1_B"),

    # Grupo C
    ("Grupo C", "4_C", "2_C"),
    ("Grupo C", "3_C", "1_C"),

    # Grupo D
    ("Grupo D", "4_D", "2_D"),
    ("Grupo D", "3_D", "1_D"),

    # Grupo E
    ("Grupo E", "4_E", "2_E"),
    ("Grupo E", "3_E", "1_E"),

    # Grupo F
    ("Grupo F", "3_F", "4_F"),
    ("Grupo F", "1_F", "2_F"),

    # Grupo G
    ("Grupo G", "4_G", "2_G"),
    ("Grupo G", "3_G", "1_G"),

    # Grupo H
    ("Grupo H", "4_H", "2_H"),
    ("Grupo H", "3_H", "1_H"),
]

# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])

# Junta com df_torneio para buscar dados dos mandantes
df_mandantes = df_torneio.rename(columns={
    "ID no Grupo": "Mandante_ID",
    "Nome do Time": "Mandante_Nome",
    "ID do Time": "Mandante_ID_Time",
    "Clube Representado": "Mandante_Clube",
    "Participante": "Mandante_Participante"
})[["Grupo", "Mandante_ID", "Mandante_Nome", "Mandante_ID_Time", "Mandante_Clube", "Mandante_Participante"]]

# Junta com df_torneio para buscar dados dos visitantes
df_visitantes = df_torneio.rename(columns={
    "ID no Grupo": "Visitante_ID",
    "Nome do Time": "Visitante_Nome",
    "ID do Time": "Visitante_ID_Time",
    "Clube Representado": "Visitante_Clube",
    "Participante": "Visitante_Participante"
})[["Grupo", "Visitante_ID", "Visitante_Nome", "Visitante_ID_Time", "Visitante_Clube", "Visitante_Participante"]]

In [4]:
# Lista dos confrontos da 2ª rodada
confrontos_2a_rodada = [
     # GRUPO A
    ("Grupo A", "1_A", "4_A"),
    ("Grupo A", "2_A", "3_A"),

    # GRUPO B
    ("Grupo B", "1_B", "4_B"),
    ("Grupo B", "2_B", "3_B"),

    # GRUPO C    
    ("Grupo C", "1_C", "4_C"),
    ("Grupo C", "2_C", "3_C"),

    # GRUPO D
    ("Grupo D", "1_D", "4_D"),
    ("Grupo D", "2_D", "3_D"),

    # GRUPO E
    ("Grupo E", "1_E", "4_E"),
    ("Grupo E", "2_E", "3_E"),

    # GRUPO F
    ("Grupo F", "4_F", "1_F"),
    ("Grupo F", "2_F", "3_F"),

    # GRUPO G    
    ("Grupo G", "1_G", "4_G"),
    ("Grupo G", "2_G", "3_G"),

    # GRUPO H
    ("Grupo H", "1_H", "4_H"),
    ("Grupo H", "2_H", "3_H"),
]

confrontos_3a_rodada = [
    # GRUPO A
    ("Grupo A", "4_A", "3_A"),
    ("Grupo A", "2_A", "1_A"),

    # GRUPO B
    ("Grupo B", "4_B", "3_B"),
    ("Grupo B", "2_B", "1_B"),

    # GRUPO C
    ("Grupo C", "4_C", "3_C"),
    ("Grupo C", "2_C", "1_C"),

    # GRUPO D
    ("Grupo D", "4_D", "3_D"),
    ("Grupo D", "2_D", "1_D"),

    # GRUPO E
    ("Grupo E", "4_E", "3_E"),
    ("Grupo E", "2_E", "1_E"),

    # GRUPO F
    ("Grupo F", "4_F", "2_F"),
    ("Grupo F", "3_F", "1_F"),

    # GRUPO G
    ("Grupo G", "4_G", "3_G"),
    ("Grupo G", "2_G", "1_G"),

    # GRUPO H
    ("Grupo H", "4_H", "3_H"),
    ("Grupo H", "2_H", "1_H"),
]

confrontos_4a_rodada = [
    # GRUPO A
    ("Grupo A", "4_A", "1_A"),
    ("Grupo A", "3_A", "2_A"),

    # GRUPO B
    ("Grupo B", "4_B", "1_B"),
    ("Grupo B", "3_B", "2_B"),

    # GRUPO C
    ("Grupo C", "4_C", "1_C"),
    ("Grupo C", "3_C", "2_C"),

    # GRUPO D
    ("Grupo D", "4_D", "1_D"),
    ("Grupo D", "3_D", "2_D"),

    # GRUPO E
    ("Grupo E", "4_E", "1_E"),
    ("Grupo E", "3_E", "2_E"),

    # GRUPO F
    ("Grupo F", "1_F", "4_F"),
    ("Grupo F", "3_F", "2_F"),

    # GRUPO G
    ("Grupo G", "4_G", "1_G"),
    ("Grupo G", "3_G", "2_G"),

    # GRUPO H
    ("Grupo H", "4_H", "1_H"),
    ("Grupo H", "3_H", "2_H"),
]


confrontos_5a_rodada = [
    # GRUPO A
    ("Grupo A", "3_A", "4_A"),
    ("Grupo A", "1_A", "2_A"),

    # GRUPO B
    ("Grupo B", "3_B", "4_B"),
    ("Grupo B", "1_B", "2_B"),

    # GRUPO C
    ("Grupo C", "3_C", "4_C"),
    ("Grupo C", "1_C", "2_C"),

    # GRUPO D
    ("Grupo D", "3_D", "4_D"),
    ("Grupo D", "1_D", "2_D"),

    # GRUPO E
    ("Grupo E", "3_E", "4_E"),
    ("Grupo E", "1_E", "2_E"),

    # GRUPO F
    ("Grupo F", "2_F", "4_F"),
    ("Grupo F", "1_F", "3_F"),

    # GRUPO G
    ("Grupo G", "3_G", "4_G"),
    ("Grupo G", "1_G", "2_G"),

    # GRUPO H
    ("Grupo H", "3_H", "4_H"),
    ("Grupo H", "1_H", "2_H"),
]


confrontos_6a_rodada = [
    # GRUPO A
    ("Grupo A", "1_A", "3_A"),
    ("Grupo A", "2_A", "4_A"),

    # GRUPO B
    ("Grupo B", "1_B", "3_B"),
    ("Grupo B", "2_B", "4_B"),

    # GRUPO C
    ("Grupo C", "1_C", "3_C"),
    ("Grupo C", "2_C", "4_C"),

    # GRUPO D
    ("Grupo D", "1_D", "3_D"),
    ("Grupo D", "2_D", "4_D"),

    # GRUPO E
    ("Grupo E", "1_E", "3_E"),
    ("Grupo E", "2_E", "4_E"),

    # GRUPO F
    ("Grupo F", "2_F", "1_F"),
    ("Grupo F", "4_F", "3_F"),

    # GRUPO G
    ("Grupo G", "1_G", "3_G"),
    ("Grupo G", "2_G", "4_G"),

    # GRUPO H
    ("Grupo H", "1_H", "3_H"),
    ("Grupo H", "2_H", "4_H"),
]


### Geração das Tabelas de Jogos das 6 Rodadas da Fase de Grupos

In [5]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos["Rodada"] = 1  # <<< ADICIONAR ESTA LINHA
df_rodada_1 = df_confrontos.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_1 = df_rodada_1.merge(df_visitantes, on=["Grupo", "Visitante_ID"])


# Transformar em DataFrame
df_confrontos_2 = pd.DataFrame(confrontos_2a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_2["Rodada"] = 2
df_rodada_2 = df_confrontos_2.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_2 = df_rodada_2.merge(df_visitantes, on=["Grupo", "Visitante_ID"])

# Transformar em DataFrame
df_confrontos_3 = pd.DataFrame(confrontos_3a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_3["Rodada"] = 3
df_rodada_3 = df_confrontos_3.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_3 = df_rodada_3.merge(df_visitantes, on=["Grupo", "Visitante_ID"])

# Criar DataFrame da 4ª rodada
df_confrontos_4 = pd.DataFrame(confrontos_4a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_4["Rodada"] = 4
df_rodada_4 = df_confrontos_4.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_4 = df_rodada_4.merge(df_visitantes, on=["Grupo", "Visitante_ID"])

# Quinta rodada
df_confrontos_5 = pd.DataFrame(confrontos_5a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_5["Rodada"] = 5
df_rodada_5 = df_confrontos_5.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_5 = df_rodada_5.merge(df_visitantes, on=["Grupo", "Visitante_ID"])

# Sexta rodada
df_confrontos_6 = pd.DataFrame(confrontos_6a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])
df_confrontos_6["Rodada"] = 6
df_rodada_6 = df_confrontos_6.merge(df_mandantes, on=["Grupo", "Mandante_ID"])
df_rodada_6 = df_rodada_6.merge(df_visitantes, on=["Grupo", "Visitante_ID"])


### Rodadas da Fase 1 da Libertadores

### Consolidação dos Confrontos da Fase 1 e Exportação para Excel

In [6]:
df_rodadas = pd.concat([
    df_rodada_1,
    df_rodada_2,
    df_rodada_3,
    df_rodada_4,
    df_rodada_5,
    df_rodada_6
], ignore_index=True)

df_rodadas.to_excel("confrontos_fase_1_libertadores.xlsx", index=False)

# Exibir os confrontos da fase 1
display(df_rodadas.head()) 

,Grupo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Mandante_Clube,Mandante_Participante,Visitante_Nome,Visitante_ID_Time,Visitante_Clube,Visitante_Participante
0,Grupo A,4_A,2_A,1,A Lenda Super Vasco F.c,117598,Carabobo,Júlio Cesar,BORGES ITAQUI F.C.,3914981,Estudiantes,Elvis
1,Grupo A,3_A,1_A,1,Dom Camillo68,20696550,Universidade Católica do Chile,Camilo,pura bucha /botafogo,18661583,Botafogo,Gabriel
2,Grupo B,4_B,2_B,1,lsauer fc,44810918,Barcelona,Sauer,Tabajara de Inhaua FC2,47543456,Independiente del Valle,Ivanildo
3,Grupo B,3_B,1_B,1,Rolo Compressor ZN,18223508,Universitário,Lucas,HS SPORTS F.C,17887202,River Plate,Henrique
4,Grupo C,4_C,2_C,1,Analove10 ITAQUI GRANDE!!,44574236,Central Córdoba,Ananias,cartola scheuer,3851966,LDU,Paulo Augusto


### Conversão dos Confrontos em JSON e Exportação para Uso no Front-End

In [7]:
# Criar lista de dicionários no formato desejado
confrontos_js = []

for _, row in df_rodadas.iterrows():
    confronto = {
        "grupo": row["Grupo"],
        "rodada": int(row["Rodada"]),
        "mandante": {
            "id": int(row["Mandante_ID_Time"]),
            "nome": row["Mandante_Nome"],
            "clube": row["Mandante_Clube"],
            "participante": row["Mandante_Participante"]
        },
        "visitante": {
            "id": int(row["Visitante_ID_Time"]),
            "nome": row["Visitante_Nome"],
            "clube": row["Visitante_Clube"],
            "participante": row["Visitante_Participante"]
        }
    }
    confrontos_js.append(confronto)

# Converter para JSON formatado
json_str = json.dumps(confrontos_js, indent=2, ensure_ascii=False)

# Salvar como arquivo JS com uma variável global
with open("confrontos_fase1_libertadores.js", "w", encoding="utf-8") as f:
    f.write("const confrontosFase1 = ")
    f.write(json_str)
    f.write(";")


In [8]:
def exibir_confrontos(df_rodadas, rodada=None, grupo=None):
    """
    Filtra e exibe os confrontos por rodada e/ou grupo.
    
    Parâmetros:
    - df_rodadas: DataFrame com todos os confrontos
    - rodada: número da rodada (int ou None para todas)
    - grupo: nome do grupo (str ou None para todos)
    
    Retorna:
    - DataFrame filtrado com as colunas relevantes
    """
    colunas = ["Rodada", "Grupo", "Mandante_Clube", "Visitante_Clube", "Mandante_Nome", "Visitante_Nome", "Mandante_Participante", "Visitante_Participante"]
    df_filtrado = df_rodadas.copy()

    df_filtrado["Rodada"] = df_filtrado["Rodada"].astype(str) + "ª Rodada"    

    if rodada is not None:
        df_filtrado = df_filtrado[df_filtrado["Rodada"] == rodada]

    if grupo is not None:
        df_filtrado = df_filtrado[df_filtrado["Grupo"] == grupo]

    return df_filtrado[colunas].sort_values(by=["Grupo", "Rodada"])


In [9]:
# # Exibir todos os confrontos da rodada 1
# display(exibir_confrontos(df_rodadas, rodada=1))

# Exibir todos os confrontos do Grupo C
display(exibir_confrontos(df_rodadas, grupo="Grupo H").head())

# # Exibir confrontos do Grupo A na rodada 2
# display(exibir_confrontos(df_rodadas, rodada=1, grupo="Grupo A"))

,Rodada,Grupo,Mandante_Clube,Visitante_Clube,Mandante_Nome,Visitante_Nome,Mandante_Participante,Visitante_Participante
14,1ª Rodada,Grupo H,San Antônio Bulo Bulo,Olimpia,Gig@ntte,MauHumor F.C.,Adriano,André
15,1ª Rodada,Grupo H,Velez Sarsfield,Peñarol,FBC Colorado,Laranjja Mecannica,Fernando,Roberto
30,2ª Rodada,Grupo H,Peñarol,San Antônio Bulo Bulo,Laranjja Mecannica,Gig@ntte,Roberto,Adriano
31,2ª Rodada,Grupo H,Olimpia,Velez Sarsfield,MauHumor F.C.,FBC Colorado,André,Fernando
46,3ª Rodada,Grupo H,San Antônio Bulo Bulo,Velez Sarsfield,Gig@ntte,FBC Colorado,Adriano,Fernando


### Coleta e Estruturação das Pontuações por Rodada dos Times do Campeonato

In [10]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(1, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    rodada_atual = api.mercado().rodada_atual
    total_rodadas = 38

    if not campeonato_comecou(api, ids_times):
        print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
        df[:] = 0
    else:
        df = pd.DataFrame()
        for nome, time_id in ids_times.items():
            pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
            df[nome] = pd.Series(pontuacoes)
        df = df.transpose()
        df.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]
        df.loc['Lider_Rodada'] = df.idxmax()
    
    return df


In [11]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)
display(df_pontuacoes.T)

,pura bucha /botafogo,BORGES ITAQUI F.C.,Dom Camillo68,A Lenda Super Vasco F.c,HS SPORTS F.C,Tabajara de Inhaua FC2,Rolo Compressor ZN,lsauer fc,TEAM LOPES 99,cartola scheuer,Grêmio imortal 37,Analove10 ITAQUI GRANDE!!,Texas Club 2025,Super Vasco f.c,Tatols Beants F.C,Fedato Futebol Clube,Real SCI,ITAQUI F. C.,E.C. Bororé,Gremiomaniasm,seralex,FC Los Castilho,TORRESMO COM PINGA,Lá do Itaqui,KING LEONN,TATITTA FC,KillerColorado,F.C. Rei Das Copas,Laranjja Mecannica,MauHumor F.C.,FBC Colorado,Gig@ntte,Lider_Rodada
Rodada 1,73.78,71.09,63.65,59.00,64.79,69.45,81.95,72.38,74.09,73.09,61.55,72.59,67.69,60.55,66.60,77.58,104.09,65.84,67.99,95.23,68.40,64.75,74.69,82.68,79.30,65.40,56.14,82.49,78.80,66.20,64.00,100.34,Real SCI
Rodada 2,73.84,71.39,67.69,88.29,94.48,87.99,74.49,79.29,73.19,86.39,99.39,76.19,85.99,96.99,73.32,77.49,75.69,71.49,86.39,59.22,72.89,95.59,86.49,82.02,93.04,59.68,103.22,82.84,96.79,102.89,72.69,86.39,KillerColorado
Rodada 3,77.12,56.40,71.20,63.70,65.12,72.95,56.00,82.17,68.42,75.10,82.42,67.40,76.92,71.40,99.55,75.42,87.02,74.02,76.92,95.55,38.22,64.47,70.10,91.77,84.70,61.70,63.90,71.62,80.10,74.30,81.52,86.80,Tatols Beants F.C
Rodada 4,88.32,99.68,136.21,86.72,108.16,94.56,76.03,104.93,109.06,77.36,122.26,119.31,79.86,90.63,94.87,90.16,99.66,76.72,63.82,85.67,77.06,78.18,98.48,81.58,84.48,90.15,99.16,74.57,63.93,119.11,78.62,79.58,Dom Camillo68
Rodada 5,76.42,67.62,103.57,81.72,83.48,91.02,53.52,66.22,87.27,80.92,63.40,86.62,71.32,85.32,83.03,81.02,82.42,54.52,80.38,86.03,69.92,87.23,86.22,86.77,72.61,58.43,69.15,67.23,74.52,69.93,69.38,73.01,Dom Camillo68
Rodada 6,77.50,109.65,78.56,93.36,82.65,106.15,68.45,108.56,106.06,98.30,88.36,104.20,119.85,100.86,86.06,92.06,103.26,56.33,103.66,78.92,66.76,86.36,98.16,82.61,99.56,73.26,77.86,70.93,49.26,107.70,92.05,78.86,Texas Club 2025
Rodada 7,98.79,92.99,119.89,94.09,98.19,113.89,85.84,87.71,127.76,95.79,91.51,106.49,98.71,96.39,105.71,104.31,94.34,102.20,91.54,93.74,99.29,94.59,96.69,90.11,87.66,86.24,95.51,79.44,79.66,92.39,94.59,90.36,TEAM LOPES 99


### Cálculo da Classificação por Grupo com Base nas Pontuações do Cartola

In [12]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

    Retorna:
    - df_resultado: classificação geral
    - df_resultado_por_grupo: dicionário com classificação separada por grupo
    """
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        grupo = confronto["Grupo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        if pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante):
            continue

        for time in [mandante, visitante]:
            if grupo not in estatisticas:
                estatisticas[grupo] = {}
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0, "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[grupo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[grupo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # Montar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [
            stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        })
        for grupo, times in estatisticas.items()
    ], ignore_index=True)

    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Vitórias", "Total Cartola", "Saldo Cartola"],
        ascending=[True, False, False, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Grupo")\
    .cumcount() + 1

    df_resultado_por_grupo = {
        grupo: df_resultado[df_resultado["Grupo"] == grupo] for grupo in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo


### Geração e Exportação da Classificação Final da Fase de Grupos

In [13]:
# Gerar a classificação da fase 1
df_resultado, df_resultado_por_grupo = classificacao_por_grupo(df_rodadas, df_pontuacoes)

# Salvar cada grupo em uma aba do Excel
with pd.ExcelWriter("classificacao_por_grupo_fase_1.xlsx") as writer:
    for grupo, df in df_resultado_por_grupo.items():
        df.to_excel(writer, sheet_name=grupo, index=False)

# Exibir a classificação geral
df_resultado_grupo_A = df_resultado[df_resultado["Grupo"] == "Grupo A"]
df_resultado_grupo_B = df_resultado[df_resultado["Grupo"] == "Grupo B"]
df_resultado_grupo_C = df_resultado[df_resultado["Grupo"] == "Grupo C"]
df_resultado_grupo_D = df_resultado[df_resultado["Grupo"] == "Grupo D"]
df_resultado_grupo_E = df_resultado[df_resultado["Grupo"] == "Grupo E"]
df_resultado_grupo_F = df_resultado[df_resultado["Grupo"] == "Grupo F"]
df_resultado_grupo_G = df_resultado[df_resultado["Grupo"] == "Grupo G"]
df_resultado_grupo_H = df_resultado[df_resultado["Grupo"] == "Grupo H"]

display(df_resultado_grupo_A, df_resultado_grupo_B, df_resultado_grupo_C, df_resultado_grupo_D, df_resultado_grupo_E, df_resultado_grupo_F, df_resultado_grupo_G, df_resultado_grupo_H)


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
2,Grupo A,Dom Camillo68,12,4,0,2,520.88,467.77,53.11,1
3,Grupo A,pura bucha /botafogo,12,4,0,2,466.98,441.24,25.74,2
1,Grupo A,BORGES ITAQUI F.C.,9,3,0,3,475.83,509.80,-33.97,3
0,Grupo A,A Lenda Super Vasco F.c,3,1,0,5,472.79,517.67,-44.88,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
5,Grupo B,Tabajara de Inhaua FC2,12,4,0,2,522.12,480.06,42.06,1
4,Grupo B,lsauer fc,12,4,0,2,513.55,487.76,25.79,2
7,Grupo B,HS SPORTS F.C,9,3,0,3,498.68,498.59,0.09,3
6,Grupo B,Rolo Compressor ZN,3,1,0,5,410.44,478.38,-67.94,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
8,Grupo C,Analove10 ITAQUI GRANDE!!,12,4,0,2,526.31,499.46,26.85,1
11,Grupo C,TEAM LOPES 99,9,3,0,3,518.09,501.43,16.66,2
10,Grupo C,Grêmio imortal 37,9,3,0,3,517.38,497.92,19.46,3
9,Grupo C,cartola scheuer,6,2,0,4,491.16,554.13,-62.97,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
15,Grupo D,Texas Club 2025,12,4,0,2,501.63,477.03,24.60,1
13,Grupo D,Super Vasco f.c,9,3,0,3,505.75,486.07,19.68,2
14,Grupo D,Tatols Beants F.C,9,3,0,3,503.43,531.60,-28.17,3
12,Grupo D,Fedato Futebol Clube,6,2,0,4,493.73,509.84,-16.11,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
19,Grupo E,Real SCI,15,5,0,1,552.14,445.08,107.06,1
16,Grupo E,Gremiomaniasm,12,4,0,2,500.62,454.82,45.80,2
18,Grupo E,E.C. Bororé,6,2,0,4,479.16,537.14,-57.98,3
17,Grupo E,ITAQUI F. C.,3,1,0,5,398.92,493.80,-94.88,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
20,Grupo F,TORRESMO COM PINGA,12,4,0,2,514.14,447.20,66.94,1
21,Grupo F,Lá do Itaqui,12,4,0,2,507.43,474.50,32.93,2
23,Grupo F,FC Los Castilho,9,3,0,3,476.58,498.67,-22.09,3
22,Grupo F,seralex,3,1,0,5,393.25,471.03,-77.78,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
27,Grupo G,KING LEONN,18,6,0,0,513.69,411.54,102.15,1
26,Grupo G,KillerColorado,9,3,0,3,469.43,467.54,1.89,2
24,Grupo G,F.C. Rei Das Copas,6,2,0,4,449.68,449.23,0.45,3
25,Grupo G,TATITTA FC,3,1,0,5,408.62,513.11,-104.49,4


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
28,Grupo H,Gig@ntte,12,4,0,2,504.98,485.52,19.46,1
31,Grupo H,Laranjja Mecannica,12,4,0,2,443.40,466.25,-22.85,2
29,Grupo H,MauHumor F.C.,9,3,0,3,540.13,485.13,55.00,3
30,Grupo H,FBC Colorado,3,1,0,5,458.26,509.87,-51.61,4


### Geração do Arquivo classificacao_fase_1.js com as Classificações por Grupo



In [14]:
# Criar estrutura em formato de dicionário para JSON/JS
classificacao_js = {}

for grupo, df in df_resultado_por_grupo.items():
    classificacao_js[grupo] = []
    for _, row in df.iterrows():
        classificacao_js[grupo].append({
            "posicao": int(row["Posição"]),
            "nome": row["Nome do Time"],
            "pontos": int(row["Pontos"]),
            "vitorias": int(row["Vitórias"]),
            "empates": int(row["Empates"]),
            "derrotas": int(row["Derrotas"]),
            "totalCartola": float(row["Total Cartola"]),
            "cartolaSofrido": float(row["Cartola Sofrido"]),
            "saldoCartola": float(row["Saldo Cartola"])
        })

# Converter para JSON formatado
json_str = json.dumps(classificacao_js, indent=2, ensure_ascii=False)

# Salvar como arquivo JS com uma variável global
with open("classificacao_por_grupo_fase_1.js", "w", encoding="utf-8") as f:
    f.write("const classificacaoFase1 = ")
    f.write(json_str)
    f.write(";")


In [15]:
def exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada, grupo=None):
    """
    Exibe os resultados de uma rodada específica, com pontuação e dados dos times.
    """

    if rodada not in df_rodadas["Rodada"].values:
        return pd.DataFrame([{
            "Grupo": grupo or "-",
            "Rodada": rodada,
            "Mandante_Nome": "-",
            "Mandante_Clube": "-",
            "Mandante_Participante": "-",
            "Mandante_Pontos": "-",
            "Visitante_Nome": "-",
            "Visitante_Clube": "-",
            "Visitante_Participante": "-",
            "Visitante_Pontos": "-",
        }])

    df_filtrado = df_rodadas[df_rodadas["Rodada"] == rodada]
    if grupo:
        df_filtrado = df_filtrado[df_filtrado["Grupo"] == grupo]

    resultados = []

    for _, row in df_filtrado.iterrows():
        grupo_ = row["Grupo"]
        mandante = row["Mandante_Nome"]
        visitante = row["Visitante_Nome"]
        mandante_clube = row["Mandante_Clube"]
        visitante_clube = row["Visitante_Clube"]
        mandante_participante = row["Mandante_Participante"]
        visitante_participante = row["Visitante_Participante"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "Grupo": grupo_,
            "Rodada": rodada,
            "Mandante_Nome": mandante,
            "Mandante_Clube": mandante_clube,
            "Mandante_Participante": mandante_participante,
            "Mandante_Pontos": pontos_mandante,
            "Visitante_Nome": visitante,
            "Visitante_Clube": visitante_clube,
            "Visitante_Participante": visitante_participante,
            "Visitante_Pontos": pontos_visitante
        })

    return pd.DataFrame(resultados)


In [16]:
# Exibir resultados da 2ª rodada
df_resultados_rodada_1 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=1)

# Exibir apenas os resultados do Grupo B na 1ª rodada
df_resultados_grupo_B = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=1, grupo="Grupo H")

# Exibir
# display(df_resultados_rodada2)
display(df_resultados_rodada_1)

,Grupo,Rodada,Mandante_Nome,Mandante_Clube,Mandante_Participante,Mandante_Pontos,Visitante_Nome,Visitante_Clube,Visitante_Participante,Visitante_Pontos
0,Grupo A,1,A Lenda Super Vasco F.c,Carabobo,Júlio Cesar,59.00,BORGES ITAQUI F.C.,Estudiantes,Elvis,71.09
1,Grupo A,1,Dom Camillo68,Universidade Católica do Chile,Camilo,63.65,pura bucha /botafogo,Botafogo,Gabriel,73.78
2,Grupo B,1,lsauer fc,Barcelona,Sauer,72.38,Tabajara de Inhaua FC2,Independiente del Valle,Ivanildo,69.45
3,Grupo B,1,Rolo Compressor ZN,Universitário,Lucas,81.95,HS SPORTS F.C,River Plate,Henrique,64.79
4,Grupo C,1,Analove10 ITAQUI GRANDE!!,Central Córdoba,Ananias,72.59,cartola scheuer,LDU,Paulo Augusto,73.09
5,Grupo C,1,Grêmio imortal 37,Deportivo Táchira,Rodrigo,61.55,TEAM LOPES 99,Flamengo,Marllon,74.09
6,Grupo D,1,Fedato Futebol Clube,Alianza Lima,Fedato,77.58,Super Vasco f.c,Libertad,Boka,60.55
7,Grupo D,1,Tatols Beants F.C,Talleres,Elemar,66.60,Texas Club 2025,São Paulo,Fumaco,67.69
8,Grupo E,1,Gremiomaniasm,Bucaramanga,Márcio,95.23,ITAQUI F. C.,Colo-Colo,Cassiano,65.84
9,Grupo E,1,E.C. Bororé,Fortaleza,Moodi,67.99,Real SCI,Racing,Diego,104.09


In [17]:
# Criar arquivo com uma aba para cada rodada contendo os resultados detalhados
from pathlib import Path

# Caminho do arquivo de saída
caminho_resultados = "resultados_fase_1.xlsx"

# Descobrir as rodadas únicas no DataFrame
rodadas_disponiveis = sorted(df_rodadas["Rodada"].unique())

with pd.ExcelWriter(caminho_resultados) as writer:
    for rodada in rodadas_disponiveis:
        df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
        nome_aba = f"Rodada {rodada}"
        df_resultados.to_excel(writer, sheet_name=nome_aba, index=False)

print(f"Arquivo salvo com sucesso: {Path(caminho_resultados).resolve()}")


Arquivo salvo com sucesso: C:\Users\ferna\Projetos\GitHub\cartola_fbc\libertadores\datasets_liberta\resultados_fase_1.xlsx


In [18]:
display(df_resultados)

,Grupo,Rodada,Mandante_Nome,Mandante_Clube,Mandante_Participante,Mandante_Pontos,Visitante_Nome,Visitante_Clube,Visitante_Participante,Visitante_Pontos
0,Grupo A,6,pura bucha /botafogo,Botafogo,Gabriel,77.50,Dom Camillo68,Universidade Católica do Chile,Camilo,78.56
1,Grupo A,6,BORGES ITAQUI F.C.,Estudiantes,Elvis,109.65,A Lenda Super Vasco F.c,Carabobo,Júlio Cesar,93.36
2,Grupo B,6,HS SPORTS F.C,River Plate,Henrique,82.65,Rolo Compressor ZN,Universitário,Lucas,68.45
3,Grupo B,6,Tabajara de Inhaua FC2,Independiente del Valle,Ivanildo,106.15,lsauer fc,Barcelona,Sauer,108.56
4,Grupo C,6,TEAM LOPES 99,Flamengo,Marllon,106.06,Grêmio imortal 37,Deportivo Táchira,Rodrigo,88.36
5,Grupo C,6,cartola scheuer,LDU,Paulo Augusto,98.30,Analove10 ITAQUI GRANDE!!,Central Córdoba,Ananias,104.20
6,Grupo D,6,Texas Club 2025,São Paulo,Fumaco,119.85,Tatols Beants F.C,Talleres,Elemar,86.06
7,Grupo D,6,Super Vasco f.c,Libertad,Boka,100.86,Fedato Futebol Clube,Alianza Lima,Fedato,92.06
8,Grupo E,6,Real SCI,Racing,Diego,103.26,E.C. Bororé,Fortaleza,Moodi,103.66
9,Grupo E,6,ITAQUI F. C.,Colo-Colo,Cassiano,56.33,Gremiomaniasm,Bucaramanga,Márcio,78.92


In [19]:
resultados_js = []

for rodada in sorted(df_rodadas["Rodada"].unique()):
    df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
    
    for _, row in df_resultados.iterrows():
        resultado = {
            "grupo": row["Grupo"],
            "rodada": int(rodada),
            "mandante": {
                "nome": row["Mandante_Nome"],
                "participante": row["Mandante_Participante"],
                "clube": row["Mandante_Clube"],
                "pontos": float(row["Mandante_Pontos"]) if row["Mandante_Pontos"] is not None else None
            },
            "visitante": {
                "nome": row["Visitante_Nome"],
                "participante": row["Visitante_Participante"],
                "clube": row["Visitante_Clube"],
                "pontos": float(row["Visitante_Pontos"]) if row["Visitante_Pontos"] is not None else None
            },
            "vencedor": (
                "mandante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] > row["Visitante_Pontos"]
                else "visitante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] < row["Visitante_Pontos"]
                else "empate" if row["Mandante_Pontos"] == row["Visitante_Pontos"] and row["Mandante_Pontos"] is not None
                else "indefinido"
            )

        }
        resultados_js.append(resultado)

# Exportar para arquivo .js
import json

with open("resultados_fase_1.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(resultados_js, indent=2, ensure_ascii=False))
    f.write(";")
